#Hadamard and concatation Mearged Model

#Build A Multi-Model

For this section, pretrained unimodels are required as well as some files which were created during that process.

To merge the models we require the output at dense layer of 300 neurons from unimodels. In the case of audio and transcript it requires some pre-processing before passing through the model, and that preprocessing requires to have all the file loaded first, so to avoid that process here some files were generated during the unimodel building process. So those files will be used to get the inputs of audio and transcript model.

While in video a sigle video file can be pre-processed and that processe implemented here.

In [ ]:
import os   # To perform directory related operations
import cv2  # To preprocess video 
# import tensorflow.compat.v1 as tf
# tf.compat.v1.disable_v2_behavior()
import numpy as np  # Datatype to store data
from tensorflow.keras import layers, Model, Input,callbacks,regularizers,models  # To build 3d CNN ML model
import matplotlib.pyplot as plt
from tensorflow import keras

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Functions and Class for processing all the files

### Class to get 300d vector of video file

In [ ]:
class video_featur_ext:
  """ This class is to convert any given video file to a 300d feature vector.
  This class store a trained model, input video is processed using 
  video_to_array function"""

  def __init__(self,path_to_model):
    """ It requires to have pre trained model of video and 
    function of video to"""

    self.model = models.load_model(path_to_video_model)
    self.inter_model = None
    # Find the layer that having output of 300d
    for i in range(len(self.model.layers)):
      output_layer = self.model.get_layer(index=i).output
      if output_layer.shape.as_list() == [None,300]:
        self.inter_model = models.Model(inputs = self.model.input,outputs = output_layer)
        break

    _,frames,width,height,channels = self.model.input_shape
    self.frames = frames
    self.width = width
    self.height = height
    self.channels = channels

  def get_vector(self,v_file):
    
    tmp_video_array = self.pre_processing_fn(v_file,(self.width,self.height),self.frames)
    
    tmp_video_array = tmp_video_array.reshape(1,self.frames,self.width,self.height,self.channels)
    return self.inter_model.predict(tmp_video_array)

  def pre_processing_fn(self,files,dia,n):
    """ Files is the path to the file and dia is output dimention and
    n is the number of frames required
    """
    cap=cv2.VideoCapture(files)  # Read the File
    total_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT) # Get Total Frames
    sample_at=int(total_frames/(n+1))  # k/n frame
    sample_at = sample_at if sample_at != 0 else 1
    cap.set(1,sample_at)  # First first k/n frame

    ret,frame=cap.read()  # Store that Frame 
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
    frame= cv2.resize(frame,dia)  # resize frame
    frame= frame.reshape((1,dia[0],dia[1],3))  
    
    for i in range(1,n):
        # Read remaning k/n th frames
#         print(i)
        s_at=sample_at*(i+1) 
        cap.set(1,s_at)
        ret,frm=cap.read()
        if not ret:
          # print("Break:",ret)
          break
        frm = cv2.cvtColor(frm, cv2.COLOR_BGR2RGB)
        frm= cv2.resize(frm,dia)
        frm= frm.reshape((1,dia[0],dia[1],3))
        frame=np.concatenate([frame,frm], axis=0)

    if frame.shape[0] < n:
        en = frame.shape[0]
        d = n - en
        frame = np.concatenate([frame,np.zeros((d,dia[0],dia[1],3))],axis=0)
    
    return frame

### Class to get 300d vector of Transcript

In [ ]:
class Text_feature_ext:
  """ This class is implemented to convert the given transcripts to the 300d vector,
  However, this class does not takes transcript file as input but only it's name.
  it uses the text file which was created earlier in unimodel named 'textTo100dV.txt' to get
  100d vector of sentence and 'conversionTxt100x300V.txt to get 100x300 matrix 
  representation of perticular trial's transcrip """

  def __init__(self,trained_text_model,text_to_100d_file,conversion_matrix_file):
    """ In this constructor it reuire a pretrained model with atleast one dense layer with
    300 neuron, a text files which were created using unimodel code."""
    self.trained_text_model = models.load_model(trained_text_model)  # Load pre-trained model
    self.text_to_100d_file = {}  # This will keep map of trial name to 100d vector
    # To read text file to create above map
    with open(text_to_100d_file,encoding="utf-8") as tvfile:
      for line in tvfile:
        file_name, vector = line.split(" ",maxsplit=1)
        vector = np.fromstring(vector, "i", sep=" ")
        self.text_to_100d_file[file_name] = vector

    # To read the conversion matrix text file    
    self.conversion_matrix_file = []
    with open(conversion_matrix_file,encoding="utf-8") as cmfile:
      for line in cmfile:
        vector = np.fromstring(line, "f", sep=" ")
        self.conversion_matrix_file.append(vector)
    self.conversion_matrix_file = np.array(self.conversion_matrix_file)

    # Creating a sub/inter - model of pretrained model to get output from that 
    # Dense layer with 300 neurons
    self.inter_model = None
    # Find the layer that having output of 300d
    for i in range(len(self.trained_text_model.layers)):
      output_layer = self.trained_text_model.get_layer(index=i).output
      if output_layer.shape.as_list() == [None,300]:
        self.inter_model = models.Model(inputs = self.trained_text_model.input,outputs = output_layer)
        break
    
  def get_vector(self,t_file_name):
    """ This will reqire just th file name without path. 
    first this function loads the 100d vector of perticular trial then,
    converts to 100x300 sized matrix using conversion matrix.
    that marix is given to that inter-model. The output of that model
    is returned"""
    text_100dV = self.text_to_100d_file[t_file_name]
    cm_input = self.conversion_matrix_file[text_100dV].T
    cm_input = cm_input.reshape(1,cm_input.shape[0],cm_input.shape[1])
    return self.inter_model.predict(cm_input)  

### Class to get 39 Micro Features

In [ ]:
class annotation_ext:
  """ This is simple class to read the micro feature csv"""
  def __init__(self,anotation_file_path):
    self.micro_expression_vector = {}
    self.micro_expression_label = {}
    with open(anotation_file_path,encoding='utf-8') as anfile:
      next(anfile) # Skipp the first row
      for line in anfile:
        file_name = line.split(",")[0]
        label = line.split(",")[-1]
        vector39F = np.fromstring(" ".join(line.split(",")[1:-1]),"i",sep=" ")
        self.micro_expression_vector[file_name] = vector39F.reshape(1,vector39F.shape[0])
        self.micro_expression_label[file_name] = label[:-1]

  def get_vector(self,file_name):
    """ When File name is provided it returns the 39d vector"""
    return self.micro_expression_vector[file_name]

### Class to get 300d vector of audio file

In [ ]:
class audio_feature_ext:
  """ This class will reqire a matrix of preprocessed audio features. and a 
  trained model of audio classification get_vector function will return 300 vector,
  A Train model should have a layer with 300 neurons """
  def __init__(self,path_to_trained_model,feature_matrix_text,truth_lie_list):
    self.model = models.load_model(path_to_trained_model)
    self.inter_model = None

    # Find the layer that having output of 300d
    for i in range(len(self.model.layers)):
      output_layer = self.model.get_layer(index=i).output
      if output_layer.shape.as_list() == [None,300]:
        self.inter_model = models.Model(inputs = self.model.input,outputs = output_layer)
        break
    self.audio_to_feat = {}
    
    with open(feature_matrix_text,encoding="utf-8") as affile:
      i = 0
      for line in affile:
        vector = np.fromstring(line, "f", sep=",")
        vector = vector[:-1]
        self.audio_to_feat[truth_lie_list[i]] = vector.reshape(1,vector.shape[0])
        i += 1
  def get_vector(self,file_name):
    tx_vector = self.audio_to_feat[file_name]

    return self.inter_model.predict(tx_vector)

### Load Files

In [ ]:
# Define Some directory
path_to_truth_video = "/content/drive/MyDrive/RealLife_DeceptionDetection/Clips/Truthful"
path_to_lie_video = "/content/drive/MyDrive/RealLife_DeceptionDetection/Clips/Deceptive"

path_to_video_model = "/content/drive/MyDrive/RealLife_DeceptionDetection/Clips/VideoModel"
path_to_audio_model = "/content/drive/MyDrive/RealLife_DeceptionDetection/audio/Best_Audio_1"
path_to_text_model  = "/content/drive/MyDrive/RealLife_DeceptionDetection/Transcription/Transcript1DCNNModel"

path_to_text100d_file = "/content/drive/MyDrive/RealLife_DeceptionDetection/Transcription/textTo100dV.txt"
path_to_text100x300cm = "/content/drive/MyDrive/RealLife_DeceptionDetection/Transcription/conversionTxt100x300V.txt"
path_to_csv_micro_exp = "/content/drive/MyDrive/RealLife_DeceptionDetection/Annotation/All_Gestures_Deceptive and Truthful.csv"

path_to_audio_features = "/content/drive/MyDrive/RealLife_DeceptionDetection/audio/audio_feature.csv"

truth_vfiles = sorted(os.listdir(path_to_truth_video))
lie_vfiles = sorted(os.listdir(path_to_lie_video))

In [ ]:
# Create 300d Vector generator objects

# Audio
audio_file_names = [ afiles[:-4]+".wav" for afiles in truth_vfiles + lie_vfiles]
audio_300dGen = audio_feature_ext(path_to_audio_model,path_to_audio_features,audio_file_names) 

# Video
video_300dGen = video_featur_ext(path_to_video_model)

# Transcript 
Text_300dGen = Text_feature_ext(path_to_text_model,path_to_text100d_file,path_to_text100x300cm)

# Annotation
Annotation_39dGen = annotation_ext(path_to_csv_micro_exp)

### Store 300-d Vectors from each model

In [ ]:
# First Create 339d Vector 

truth_339d = []
lie_339d = []

for id in truth_vfiles:
  v_file_path = os.path.join(path_to_truth_video,id)
  t_file_name = id[:-4]+".txt"
  a_file_name = id[:-4]+".wav"
  vector_1 = video_300dGen.get_vector(v_file_path)
  vector_2 = Text_300dGen.get_vector(t_file_name)
  vector_3 = audio_300dGen.get_vector(a_file_name)
  vector_4 = Annotation_39dGen.get_vector(id)
  vector_h = vector_1 * vector_2 * vector_3
  vector_h_c= np.concatenate([vector_h,vector_4], axis=1)
  
  truth_339d.append(vector_h_c)
truth_339d = np.array(truth_339d)


for id in lie_vfiles:
  v_file_path = os.path.join(path_to_lie_video,id)
  t_file_name = id[:-4]+".txt"
  a_file_name = id[:-4]+".wav"
  vector_1 = video_300dGen.get_vector(v_file_path)
  vector_2 = Text_300dGen.get_vector(t_file_name)
  vector_3 = audio_300dGen.get_vector(a_file_name)
  vector_4 = Annotation_39dGen.get_vector(id)
  vector_h = vector_1 * vector_2 * vector_3
  vector_h_c= np.concatenate([vector_h,vector_4], axis=1)
  
  lie_339d.append(vector_h_c)
lie_339d = np.array(lie_339d)

In [ ]:
lie_339d.shape

(61, 1, 339)

In [ ]:
truth_339d = truth_339d.reshape(truth_339d.shape[0],truth_339d.shape[2])
lie_339d = lie_339d.reshape(lie_339d.shape[0],lie_339d.shape[2])

### Split Train and Test Data

In [ ]:
n = int(truth_339d.shape[0]*0.7)
x_train_truth = truth_339d[:n]
y_train_truth = np.ones(shape = (x_train_truth.shape[0],1))

x_train_lie = lie_339d[:n]
y_train_lie = np.zeros(shape = (x_train_lie.shape[0],1))

x_train = np.concatenate([x_train_truth,x_train_lie],axis=0)
y_train = np.concatenate([y_train_truth,y_train_lie],axis=0)


x_test_truth = truth_339d[n:]
y_test_truth = np.ones(shape = (x_test_truth.shape[0],1))

x_test_lie = lie_339d[n:]
y_test_lie = np.zeros(shape = (x_test_lie.shape[0],1))

x_test = np.concatenate([x_test_truth,x_test_lie],axis=0)
y_test = np.concatenate([y_test_truth,y_test_lie],axis=0)

# Shuffle Training data
sd = 1112
rnd = np.random.RandomState(sd)
rnd.shuffle(y_train)

sd = 1112
rnd = np.random.RandomState(sd)
rnd.shuffle(x_train)

# Shuffle Testing data
sd = 1114
rnd = np.random.RandomState(sd)
rnd.shuffle(y_test)

sd = 1114
rnd = np.random.RandomState(sd)
rnd.shuffle(x_test)

### Build MLP model

In [ ]:
# Following "cb" Function is callback function to early stop the training based on validation set accuracy
cb = keras.callbacks.EarlyStopping(monitor="val_acc",
    min_delta=0.01,
    patience=200,
    verbose=0,
    mode="max",
    baseline=None,
    restore_best_weights=True)


# Following is the 1D CNN for finding features from the text word2vectors
# Model is build using daisy chain layers;

int_sequences_input = keras.Input(shape=(x_train.shape[1]))

x = layers.Dense(1024,activation="relu")(int_sequences_input)
x = layers.Dropout(0.5)(x)

preds = layers.Dense(2, activation="softmax")(x)

# Combine all the layers into a model
model = keras.Model(int_sequences_input, preds)

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 339)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              348160    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2050      
Total params: 350,210
Trainable params: 350,210
Non-trainable params: 0
_________________________________________________________________


### Train Model

In [ ]:
# Model is compiled and loss is considered as sparse Categorical Cross Entropy
# To calculate model weights RMSProp is used

model.compile(
    loss="sparse_categorical_crossentropy", optimizer='rmsprop', metrics=["acc"]
)

""" Model is trained using training data; Validation data is used to check the model's accuracy on test data
    Note:- these validation data is not used in weight update""" 
history = model.fit(x_train, y_train,epochs=1000,validation_data=(x_test, y_test),callbacks=[cb])

Train on 84 samples, validate on 37 samples
Epoch 1/1000
84/84 [==============================] - 0s 3ms/sample - loss: 27.2244 - acc: 0.5000 - val_loss: 2.7181 - val_acc: 0.4054
Epoch 2/1000
84/84 [==============================] - 0s 201us/sample - loss: 20.8272 - acc: 0.6310 - val_loss: 0.9985 - val_acc: 0.5946
Epoch 3/1000
84/84 [==============================] - 0s 154us/sample - loss: 6.1904 - acc: 0.8095 - val_loss: 8.2844 - val_acc: 0.5405
Epoch 4/1000
84/84 [==============================] - 0s 156us/sample - loss: 4.6734 - acc: 0.8690 - val_loss: 1.4844 - val_acc: 0.5135
Epoch 5/1000
84/84 [==============================] - 0s 199us/sample - loss: 12.7972 - acc: 0.7857 - val_loss: 6.9712 - val_acc: 0.5676
Epoch 6/1000
84/84 [==============================] - 0s 155us/sample - loss: 6.2454 - acc: 0.8571 - val_loss: 4.8129 - val_acc: 0.5405
Epoch 7/1000
84/84 [==============================] - 0s 150us/sample - loss: 3.2557 - acc: 0.8810 - val_loss: 2.5822 - val_acc: 0.5676


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Epoch 8/1000
84/84 [==============================] - 0s 156us/sample - loss: 1.5948 - acc: 0.9286 - val_loss: 3.5775 - val_acc: 0.5946
Epoch 9/1000
84/84 [==============================] - 0s 155us/sample - loss: 2.2764 - acc: 0.9286 - val_loss: 1.9031 - val_acc: 0.6757
Epoch 10/1000
84/84 [==============================] - 0s 152us/sample - loss: 10.3686 - acc: 0.8333 - val_loss: 2.2314 - val_acc: 0.7297
Epoch 11/1000
84/84 [==============================] - 0s 157us/sample - loss: 6.9313 - acc: 0.8810 - val_loss: 3.6264 - val_acc: 0.5676
Epoch 12/1000
84/84 [==============================] - 0s 143us/sample - loss: 1.7945 - acc: 0.9048 - val_loss: 1.0645 - val_acc: 0.7027
Epoch 13/1000
84/84 [==============================] - 0s 135us/sample - loss: 0.8127 - acc: 0.9286 - val_loss: 4.5874 - val_acc: 0.5946
Epoch 14/1000
84/84 [==============================] - 0s 159us/sample - loss: 2.0849 - acc: 0.8929 - val_loss: 3.6635 - val_acc: 0.6216
Epoch 15/1000
84/84 [=====================

In [ ]:
# Model prediction accuracy on the test data (it's same as validation data)
print("Accuracy on Test Data:", model.evaluate(x_test,y_test))

Accuracy on Test Data: [1.4424756346522152, 0.8108108]


In [ ]:
model.save("/content/drive/MyDrive/RealLife_DeceptionDetection/MultiModelConcatinatedModel_HM")

In [ ]:
# model.save("MyModelfile")

In [ ]:
model.save_weights("/content/drive/MyDrive/RealLife_DeceptionDetection/MultimodelConvatinated_weights/weightsC_HM")